In [7]:
import math
def spikes_between(arr, start, end):
    return arr[np.where((start<=arr) & (arr<end))[0]]

# def spikes_between(arr, start, end):
#     return arr[np.where((start<=arr) & (arr<end))[0]]#arr[np.where(start<=arr)][np.where(arr[np.where(start<=arr)]<end)]
# def count_between(arr, start, end):
#     # print(type(arr))
#     return len(np.where((start<=arr) & (arr<end))[0])#<end][np.where(arr[np.where(start<=arr)]<end)])
def between(arr, start, end):
    return arr[np.where(start<=arr)][np.where(arr[np.where(start<=arr)]<end)]
def count_between(arr, start, end):
    return len(arr[np.where(start<=arr)][np.where(arr[np.where(start<=arr)]<end)])


def count_output_spikes_per_inputs(spikeTrain, number_of_input_data ,total_synaptic_delay = 0, timesteps =1, number_of_steps = 10, inputs_interval = 0, offset = 0):
    # All the times must be in using dimension: uSecond or ... considering the `timestamp_coefficient` which is used in FPGA_Pop_SSA
    data_length = timesteps*number_of_steps
    start_base = data_length+offset+total_synaptic_delay+inputs_interval
    s=[]
    e=[]
    s.append(0)
    e.append(data_length+offset+inputs_interval//2+total_synaptic_delay)
#     print(f'Strat: {s[0]}==> end: {e[0]}')

    for i in range(1, number_of_input_data):
        # s.append(i*data_length+offset+i*(inputs_interval)+total_synaptic_delay)
        # e.append((i+1)*data_length+offset+(i+1)*(inputs_interval)+total_synaptic_delay)
        s.append(e[i-1])
        e.append(s[i]+data_length+inputs_interval)
#         print(f'Strat: {s[i]}==> end: {e[i]}')
    
    n_output_neurons = len(spikeTrain)
    print(n_output_neurons)
    n_spikes_per_input_as_output = torch.zeros(n_output_neurons, number_of_input_data)
    for i in range(number_of_input_data):
        for j in range(n_output_neurons):
            # print(spikeTrain[j])
            # print(s[i], e[i])
            # print(len(spikes_between(np.array(spikeTrain[j]), s[i], e[i])))
            n_spikes_per_input_as_output[j, i]=count_between(np.array(spikeTrain[j]), s[i], e[i]) #len(spikes_between(np.array(spikeTrain[j]), s[i], e[i]))
            # break
    output_spike_count_matrix = n_spikes_per_input_as_output
    return output_spike_count_matrix # retruns a matrix=> one row per output, each column shows the number of spikes caused per input 

def maxSpike_accuracy(true_labels, output_spike_count_matrix):

    print(f'n_spikes_per_input_as_output = {output_spike_count_matrix}')
    print(torch.argmax(output_spike_count_matrix,0))
    print(true_labels)

    print(torch.sum(torch.argmax(output_spike_count_matrix,0) == torch.tensor(true_labels)))
    _, predicted = torch.max(output_spike_count_matrix, 0)
    n_correct = (predicted == torch.tensor(targets[:n_input_samples])).sum().item()
    print(n_correct)



    acc = 100.0 * n_correct / n_input_samples
    print(f'Accuracy of the network on the {n_input_samples} test images: {acc} %')
    return


def FPGA_Pop_SSA(dataLoader, input_size = [128, 128, 2], timesteps =1, number_of_steps = 10, inputs_interval = 0, offset = 0, timestamp_coefficient = 1000): 
    # timestamp_coefficient = 1000, for nSec to uSec and etc.
    # Here since the input data is from the DVS sensor and it's already timed (timestamps), thus we don't need to set the timesteo and number of steps
    # input_size is the sensor size

    spikes_time_source_array=[[] for _ in range(np.prod(input_size))]
    current_time = offset
    targets = []
    data_as_list = []
    for data, target in dataLoader:
        
        targets.append(target)
        data = data.squeeze(0)
        data_as_list.append(data)
        # data[:,2]+= inputs_interval
        if len(data)>0:
            pre_timestamp = data[0,2].item()#np.ceil(data[0,2].item()/timestamp_coefficient).astype(np.int32)#data[0,2]#np.ceil(data[0,2].item()//timestamp_coefficient).astype(np.int32)
            spikes_time_source_array[((data[0,1] << 7) + (data[0,3] << 14)   + (data[0,0])).to(int)].append(current_time)#np.ceil(data[0,2].item()//timestamp_coefficient).astype(np.int32))
            for e in data[1:]:
                post_timestamp = e[2].item()#np.ceil(e[2].item()/timestamp_coefficient).astype(np.int32)#e[2].item()#np.ceil(e[2].item()//timestamp_coefficient).astype(np.int32)
                time_gap = np.ceil((post_timestamp - pre_timestamp)/timestamp_coefficient).astype(np.int32)
                current_time +=time_gap
                spikes_time_source_array[((e[1] << 7) + (e[3] << 14)   + (e[0])).to(int)].append(current_time)
                pre_timestamp = post_timestamp
                # current_time+=event_interval
            current_time+=inputs_interval

    return spikes_time_source_array, data_as_list, targets, current_time

def FPGA_Pop_SSA_2(dataLoader, input_size = [128, 128, 2], timesteps =1, number_of_steps = 10, inputs_interval = 0, offset = 0):#, timestamp_coefficient = 1000): 
    # timestamp_coefficient = 1000, for nSec to uSec and etc.
    # Here since the input data is from the DVS sensor and it's already timed (timestamps), thus we don't need to set the timesteo and number of steps
    # But- they are from different files and the timestamps are not ordered, so the only thing to keep the temporal aspect of the event is to keep inter_spikes times (time_gap parameter used for)
    # input_size is the sensor size

    spikes_time_source_array=[[] for _ in range(np.prod(input_size))]
    current_time = offset
    targets = []
    data_as_list = []
    for data, target in dataLoader:
        
        targets.append(target)
        data = data.squeeze(0)
        data_as_list.append(data)
        # data[:,2]+= inputs_interval
        if len(data)>0:
            pre_timestamp = data[0,2].item()#np.ceil(data[0,2].item()/timestamp_coefficient).astype(np.int32)#data[0,2]#np.ceil(data[0,2].item()//timestamp_coefficient).astype(np.int32)
            spikes_time_source_array[((data[0,1] << 7) + (data[0,3] << 14)   + (data[0,0])).to(int)].append(current_time)#np.ceil(data[0,2].item()//timestamp_coefficient).astype(np.int32))
            for e in data[1:]:
                post_timestamp = e[2].item()#np.ceil(e[2].item()/timestamp_coefficient).astype(np.int32)#e[2].item()#np.ceil(e[2].item()//timestamp_coefficient).astype(np.int32)
                time_gap = post_timestamp - pre_timestamp
                current_time +=time_gap
                spikes_time_source_array[((e[1] << 7) + (e[3] << 14)   + (e[0])).to(int)].append(current_time)
                pre_timestamp = post_timestamp
                # current_time+=event_interval
            current_time+=inputs_interval

    return spikes_time_source_array, data_as_list, targets, current_time


def plot_spikes_count_as_frame(output_spike_count_matrix, index, data_as_list = None):

    fig = plt.figure()
    fig.add_subplot(1,2,1)
    dim = int(math.sqrt(len(output_spike_count_matrix)/2))
    plt.imshow(output_spike_count_matrix[:,index].reshape(2, dim,dim)[1].T)#, cmap='jet')#(dim,dim, 2)[:,:,0])
    fig.add_subplot(1,2,2)
    plt.imshow(output_spike_count_matrix[:,index].reshape(2, dim,dim)[0].T)#, cmap='jet')#(128,128, 2)[:,:,1])
        
    if data_as_list!=None:
        fig = plt.figure()
        d = data_as_list[index]
        frame = torch.zeros(2, 128, 128)
        pos = d[torch.where(d[:,3] ==1)]
        neg = d[torch.where(d[:,3] ==0)]
        frame[0, pos[:,0].to(int), pos[:,1].to(int)] = 1
        frame[1, neg[:,0].to(int), neg[:,1].to(int)] = 1
        fig.add_subplot(1,2,1)
        plt.imshow(frame[0])
        fig.add_subplot(1,2,2)
        plt.imshow(frame[1])    
    return

## Dataloaders

In [11]:
import warnings
warnings.filterwarnings('ignore')
import my_data_utils
import torch
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, random_split


# dataset_all = my_data_utils.AprovisDataSet(src_path='Data', same_size_classes=False,  verbose=False, temporal_scale_factor=1000, time_window=1e2)# temporal_scale_factor = 1000: uSec to mSec, time_window = 100: each 100 mSec is one sample
# PacmanTooBigToPlace: SpikeSourceArrayVertex_1 will not fit on any possible Chip 
# the reason is that [SpikeSourceArrayVertex_1(0:32767)] requires 543207176.0 bytes but a Chip only has 123469792 bytes Lowering max_core_per_chip may resolve this.

dataset_all = my_data_utils.AprovisDataSet(src_path='Data', same_size_classes=False,  verbose=False, temporal_scale_factor=1000, time_window=1e3)# temporal_scale_factor = 1000: uSec to mSec, time_window = 1000: each 1000 mSec is one sample
                                                                                                                                                  # # temporal_scale_factor = 1: uSec, time_window = 1e6: each 1e6 uSec is one sample

print(len(dataset_all))                                                                                                                        
print(dataset_all.n_total_samples_in_cls)

def split_train(full_dataset, train_fraction,transform):
    # print(train_data_folder)
    # data_train = full_dataset#ImageFolder(train_data_folder, transform)
    fractions = [int(train_fraction*len(full_dataset)),len(full_dataset)-int(train_fraction*len(full_dataset))]
    print(len(full_dataset))
    print(f'fractions: {fractions}')
    train, test = random_split(full_dataset, fractions, generator=torch.Generator().manual_seed(42))
    return train, test

train_set, test_set = split_train(full_dataset = dataset_all, train_fraction=.75, transform=None)
batch_size = 1
trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=False)
testloader = DataLoader(test_set, batch_size=batch_size, shuffle=False)


same_size_classes: False
n_total_samples_in_cls 1: {'sea': 288, 'gro': 279}
n_total_samples_in_cls 2: {'sea': 288, 'gro': 279}
567
{'sea': 288, 'gro': 279}
567
fractions: [425, 142]


## Using the same connection list that is used for connecting the FPGA Population to the first population (https://github.com/ntouev/ev_snn_percept/blob/main/spinnaker/scripts/rt-online_ht_snn.py)

In [12]:
RETINA_DIM = 128
# xy dimension is RETINA_DIM/SCALEDOWN x RETINA_DIM/SCALEDOWN
SCALEDOWN = 4
# planar dimension is THETA_NUM x R_NUM
THETA_NUM = 90
R_NUM = 60   

################################################

X_NUM = round(RETINA_DIM/SCALEDOWN)
Y_NUM = round(RETINA_DIM/SCALEDOWN)
P_NUM = 2

connections_ssa_xy = []
for ret_p in range(P_NUM):
    for ret_y in range(RETINA_DIM):
        for ret_x in range(RETINA_DIM):
#             if (ret_x >= L) and (ret_x <= RETINA_DIM - L - 1):    
#                 ssa_xy_weight_mod = SSA_XY_WEIGHT * G
#             else:
#                 ssa_xy_weight_mod = SSA_XY_WEIGHT

            tup = (ret_x + RETINA_DIM*ret_y + RETINA_DIM*RETINA_DIM*ret_p, 
                   ret_x//SCALEDOWN + X_NUM*(ret_y//SCALEDOWN) + X_NUM*Y_NUM*ret_p,
                   10,#ssa_xy_weight_mod,
                   1)#SSA_XY_DELAY) 
            connections_ssa_xy.append(tup)


## Converting the dataLoader into SpikeSourceArray: It must be like what we will have from the `FPGA population` (DVS inputs)

In [13]:
SSA_as_output_for_test, data_as_list, targets, time = FPGA_Pop_SSA_2(testloader, [128, 128, 2], inputs_interval= 0)# inputs_interval= 0 timestep (here is mSce): to have the consider as a stream
time

141364

In [14]:
import pyNN.spiNNaker as p
import pyNN.utility.plotting as plot
# import matplotlib.pyplot as plt

p.setup(1.0)

ssa = p.Population(RETINA_DIM*RETINA_DIM*P_NUM, p.SpikeSourceArray(spike_times=SSA_as_output_for_test))

pop_xy = p.Population(X_NUM*Y_NUM*P_NUM,
                      p.IF_curr_exp,
                      label = 'pop_xy')

pop_out = p.Population(512,
                          p.IF_curr_exp,
                          label = 'pop_out')


ssa.record(['spikes'])
pop_xy.record(['spikes'])
pop_out.record(['spikes'])

p.Projection(ssa, 
             pop_xy, 
             p.FromListConnector(connections_ssa_xy),
             p.StaticSynapse())

connections_list_from_file=p.FromFileConnector('ConnectionFile_w_fc1_2048_512_32_32_ANN_BCE_One_Layer_84_testAccuracy.txt')

xy_out_prj = p.Projection(pop_xy, 
             pop_out, 
             connections_list_from_file,
             p.StaticSynapse())



#######################################################################################################
simtime = time
p.run(simtime)


2023-07-03 10:13:53 INFO: Read configs files: /home/bbpnrsoa/sPyNNakerGit/SpiNNUtils/spinn_utilities/spinn_utilities.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNMachine/spinn_machine/spinn_machine.cfg, /home/bbpnrsoa/sPyNNakerGit/PACMAN/pacman/pacman.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNMan/spinnman/spinnman.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg, /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg, /home/bbpnrsoa/.spynnaker.cfg
2023-07-03 10:13:53 INFO: Will search these locations for binaries: /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/model_binaries : /home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/common_model_binaries
2023-07-03 10:13:53 INFO: Setting hardware timestep as 1000 microseconds based on simulation time step of 1000 and timescale factor of 1.0


['/home/bbpnrsoa/sPyNNakerGit/SpiNNUtils/spinn_utilities/spinn_utilities.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNMachine/spinn_machine/spinn_machine.cfg', '/home/bbpnrsoa/sPyNNakerGit/PACMAN/pacman/pacman.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNMan/spinnman/spinnman.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg', '/home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg', '/home/bbpnrsoa/.spynnaker.cfg']


2023-07-03 10:13:53 WARNING: Danger of SpikeSourceArray sending too many spikes at the same time. For example at time 2149, 172 spikes will be sent
2023-07-03 10:13:53 INFO: Starting execution process
2023-07-03 10:13:53 INFO: Simulating for 141364 1.0 ms timesteps using a hardware timestep of 1000 us
2023-07-03 10:13:54 INFO: SpYNNakerNeuronGraphNetworkSpecificationReport skipped as cfg Reports:write_network_graph is False
2023-07-03 10:13:54 INFO: Network Specification report took 0:00:00.000500 
2023-07-03 10:13:54 INFO: Splitter reset took 0:00:00.000012 
Adding Splitter selectors where appropriate
|0%                          50%                         100%|
2023-07-03 10:13:54 INFO: Spynnaker splitter selector took 0:00:00.012745 
Adding delay extensions as required
|0%                          50%                         100%|
2023-07-03 10:13:54 INFO: DelaySupportAdder took 0:00:00.012542 
Partitioning Graph
|0%                          50%                         100%|
2023-0

141364.0

## Extracting the output spikeTrains (per output neurons) and using the `count_output_spikes_per_inputs` function for counting the number of generated spikes per each input samples

In [19]:
output_size = 1
output_spike_trains=[pop_out.get_data("spikes").segments[0].spiketrains[i][:] for i in range(output_size)]
# count_output_spikes_per_inputs(spikeTrain=output_spike_trains, number_of_input_data= len(testloader), total_synaptic_delay=1, timesteps=1, number_of_steps=data_lenght, inputs_interval=5000)
data_lenght = 1000 # time_window = 100 mSec
output_spike_count_matrix = count_output_spikes_per_inputs(spikeTrain=output_spike_trains, number_of_input_data= len(testloader), total_synaptic_delay=1, timesteps=1, number_of_steps=data_lenght, inputs_interval=0)
# timesteps=1, number_of_steps=1e6 considering the time_window in daraset to have the correct lenght of each input
# sample_interval in uSeconds


1


In [20]:
threshold = 0
for i in range(len(testloader)):
    print(f'True label: {targets[i].item()}, prediction: {int(output_spike_count_matrix[0][i]>threshold)}, number of spikes: {output_spike_count_matrix[0][i].item()}' )

True label: 1, prediction: 1, number of spikes: 88.0
True label: 0, prediction: 0, number of spikes: 0.0
True label: 1, prediction: 1, number of spikes: 254.0
True label: 1, prediction: 1, number of spikes: 1.0
True label: 0, prediction: 0, number of spikes: 0.0
True label: 1, prediction: 1, number of spikes: 105.0
True label: 1, prediction: 1, number of spikes: 165.0
True label: 0, prediction: 0, number of spikes: 0.0
True label: 0, prediction: 1, number of spikes: 2.0
True label: 1, prediction: 1, number of spikes: 166.0
True label: 0, prediction: 0, number of spikes: 0.0
True label: 0, prediction: 0, number of spikes: 0.0
True label: 0, prediction: 1, number of spikes: 2.0
True label: 1, prediction: 1, number of spikes: 76.0
True label: 1, prediction: 1, number of spikes: 96.0
True label: 1, prediction: 1, number of spikes: 254.0
True label: 0, prediction: 1, number of spikes: 1.0
True label: 0, prediction: 1, number of spikes: 54.0
True label: 1, prediction: 1, number of spikes: 73

## Check accuracy for BCE: it will only have output when the input is peridicted as `land` 

In [21]:
((output_spike_count_matrix[0]>0).to(int)==torch.tensor(targets))

tensor([ True,  True,  True,  True,  True,  True,  True,  True, False,  True,
         True,  True, False,  True,  True,  True, False, False,  True,  True,
         True, False,  True, False,  True, False, False, False, False, False,
         True,  True, False,  True,  True,  True,  True, False,  True, False,
        False, False, False,  True, False,  True, False,  True, False, False,
        False,  True, False,  True, False,  True, False,  True,  True,  True,
         True, False, False, False,  True, False,  True, False,  True, False,
         True, False, False,  True,  True, False,  True, False,  True,  True,
         True,  True,  True,  True,  True,  True, False, False, False,  True,
         True,  True, False,  True, False,  True, False, False,  True, False,
         True,  True,  True, False,  True,  True, False,  True, False,  True,
         True,  True, False,  True,  True, False, False,  True, False, False,
         True,  True, False,  True,  True, False,  True,  True, 

## We can deifine a threshold for the `number of output spikes` 

In [22]:
n_input_samples = len(testloader)
thresholds = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
for threshold in thresholds:
    n_correct = sum((output_spike_count_matrix[0]>threshold).to(int)==torch.tensor(targets))
    acc = 100.0 * n_correct / n_input_samples
    print(f'Accuracy of the network on the {n_input_samples} test with threshold = {threshold}: {acc} %')


Accuracy of the network on the 142 test with threshold = 0: 58.45070266723633 %
Accuracy of the network on the 142 test with threshold = 10: 68.30986022949219 %
Accuracy of the network on the 142 test with threshold = 20: 71.12676239013672 %
Accuracy of the network on the 142 test with threshold = 30: 73.94366455078125 %
Accuracy of the network on the 142 test with threshold = 40: 74.64788818359375 %
Accuracy of the network on the 142 test with threshold = 50: 73.94366455078125 %
Accuracy of the network on the 142 test with threshold = 60: 73.94366455078125 %
Accuracy of the network on the 142 test with threshold = 70: 73.23943328857422 %
Accuracy of the network on the 142 test with threshold = 80: 71.83098602294922 %
Accuracy of the network on the 142 test with threshold = 90: 69.71830749511719 %
Accuracy of the network on the 142 test with threshold = 100: 64.08451080322266 %
